# Dogs breeds

https://youtu.be/JNxcznsrRb8?t=1h31m8s

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
torch.cuda.set_device(0)

In [4]:
PATH = "/home/embsysa/fastai/pytorch_classifiers-master/data/"
sz = 224
arch = resnet34
bs = 24

## Initial exploration

In [5]:
import os
data_path = "/home/embsysa/fastai/pytorch_classifiers-master/data"
os.chdir(data_path)

In [6]:
!ls {PATH}

label_csv_name = 'Major_Graphics_labels.csv'
label_df = pd.read_csv(label_csv_name)


 all_mask.zip				  Major_Background_labels.csv
 Background_color_allmultilabel.csv	  Major_Graphics_labels.csv
 Background_color_allmultilabel.csv.csv   Major_Graphics_labels.csv.csv
 Backup					  Major_Graphics_labels_final.csv
'change file type.ipynb'		  old_Major_Background_labels.csv
 Emotion_labels.csv			  test
 Emotion_labels.csv.csv			  tmp
'Final data Folders'			  train
'Final data Folders.zip'		  train_with_mask


In [7]:
images = [each.split(".jpg")[0] for each in os.listdir(data_path+"/train")]

In [8]:
label_df.head()

,Image_index,Major_Graphics
0,28,99
1,40,1
2,698,2
3,769,1
4,786,5


In [9]:
label_df = label_df[label_df.Image_index.isin(images)]
label_df.to_csv(label_csv_name+'.csv', index=False)

In [10]:
label_csv_name

'Major_Graphics_labels.csv'

In [11]:
label_csv = f'{PATH}'+label_csv_name+'.csv'
n = len(label_df)# header is not counted (-1)
val_idxs = get_cv_idxs(n) # random 20% data for validation set

In [12]:
label_df.columns = label_df.columns.str.replace("-","_")
label_df.head()
#label_df.columns = ['id', 'color']

label_df.shape

(6324, 2)

In [13]:
#label_df.pivot_table(index="color", aggfunc=len).sort_values('id', ascending=False)
import numpy as np
np.max(val_idxs)

6321

In [14]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.2)
data = ImageClassifierData.from_csv(PATH, 'train', label_csv, test_name='test', # we need to specify where the test set is if you want to submit to Kaggle competitions
                                   val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

fn = PATH + data.trn_ds.fnames[0];
print(fn)

data.trn_ds.fnames[0]

img = PIL.Image.open(fn); img

img.size

/home/embsysa/fastai/pytorch_classifiers-master/data/train/28.jpg


(512, 512)

In [15]:
len(data.val_y)

1264

In [16]:
size_d = {k: PIL.Image.open(PATH + k).size for k in data.trn_ds.fnames}

In [17]:
row_sz, col_sz = list(zip(*size_d.values()))

In [18]:
row_sz = np.array(row_sz); col_sz = np.array(col_sz)

## Initial model

In [19]:
def get_data(sz, bs): # sz: image size, bs: batch size
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.2)
    data = ImageClassifierData.from_csv(PATH, 'train', label_csv, test_name='test',
                                       val_idxs=val_idxs, suffix='.jpg', tfms=tfms,bs = bs)
    
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/13
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/37
    return data if sz > 300 else data.resize(256, 'tmp') # Reading the jpgs and resizing is slow for big images, so resizing them all to 340 first saves time

#Source:   
#    def resize(self, targ, new_path):
#        new_ds = []
#        dls = [self.trn_dl,self.val_dl,self.fix_dl,self.aug_dl]
#        if self.test_dl: dls += [self.test_dl, self.test_aug_dl]
#        else: dls += [None,None]
#        t = tqdm_notebook(dls)
#        for dl in t: new_ds.append(self.resized(dl, targ, new_path))
#        t.close()
#        return self.__class__(new_ds[0].path, new_ds, self.bs, self.num_workers, self.classes)
#File:      ~/fastai/courses/dl1/fastai/dataset.py

### Precompute

In [20]:
data = get_data(sz, bs)

In [21]:
learn = ConvLearner.pretrained(arch, data, precompute=False)

In [22]:
learn.unfreeze()

# set requires_grads to be True for all layers, so they can be updated

learning_rate = [0.01, 0.01, 0.1]
# learning rate is set so that deepest third of layers have a rate of 0.001, # middle layers have a rate of 0.01, and final layers 0.1.


learn.precompute = False

In [ ]:
learn.fit(1e-2, 10)

epoch      trn_loss   val_loss   accuracy                   
    0      2.199656   2.823254   0.326741  
Validation:  13%|█▎        | 7/53 [00:00<00:04, 10.66it/s]  

### Augment

In [ ]:
from sklearn import metrics

In [ ]:
data = get_data(sz, bs)

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [ ]:
learn.fit(1e-2, 2)

In [ ]:
learn.precompute = False

In [ ]:
learn.fit(1e-2, 5, cycle_len=1)

In [ ]:
learn.save('224_pre')

In [ ]:
learn.load('224_pre')

## Increase size

In [ ]:
# Starting training on small images for a few epochs, then switching to bigger images, and continuing training is an amazingly effective way to avoid overfitting.

# http://forums.fast.ai/t/planet-classification-challenge/7824/96
# set_data doesn’t change the model at all. It just gives it new data to train with.
learn.set_data(get_data(299, bs)) 
learn.freeze()

#Source:   
#    def set_data(self, data, precompute=False):
#        super().set_data(data)
#        if precompute:
#            self.unfreeze()
#            self.save_fc1()
#            self.freeze()
#            self.precompute = True
#        else:
#            self.freeze()
#File:      ~/fastai/courses/dl1/fastai/conv_learner.py

In [23]:
learn.summary()

OrderedDict([('Conv2d-1',
              OrderedDict([('input_shape', [-1, 3, 224, 224]),
                           ('output_shape', [-1, 64, 112, 112]),
                           ('trainable', True),
                           ('nb_params', tensor(9408))])),
             ('BatchNorm2d-2',
              OrderedDict([('input_shape', [-1, 64, 112, 112]),
                           ('output_shape', [-1, 64, 112, 112]),
                           ('trainable', True),
                           ('nb_params', tensor(128))])),
             ('ReLU-3',
              OrderedDict([('input_shape', [-1, 64, 112, 112]),
                           ('output_shape', [-1, 64, 112, 112]),
                           ('nb_params', 0)])),
             ('MaxPool2d-4',
              OrderedDict([('input_shape', [-1, 64, 112, 112]),
                           ('output_shape', [-1, 64, 56, 56]),
                           ('nb_params', 0)])),
             ('Conv2d-5',
              OrderedDict([('input_shape',

In [24]:
learn.fit(1e-2, 3, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                   
    0      1.331      1.311638   0.587361  
    1      1.294006   1.304416   0.579926                   
    2      1.305864   1.31023    0.576739                   


[1.3102298772404692, 0.576739261173151]

Validation loss is much lower than training loss. This is a sign of underfitting. Cycle_len=1 may be too short. Let's set cycle_mult=2 to find better parameter.

In [25]:
learn.unfreeze()

# set requires_grads to be True for all layers, so they can be updated

learning_rate = [1e-5, 1e-5, 1e-4]
# learning rate is set so that deepest third of layers have a rate of 0.001, # middle layers have a rate of 0.01, and final layers 0.1.

In [26]:
# When you are under fitting, it means cycle_len=1 is too short (learning rate is getting reset before it had the chance to zoom in properly).
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2) # 1+2+4 = 7 epochs

epoch      trn_loss   val_loss   accuracy                   
    0      1.277813   1.302588   0.588423  
    1      1.321117   1.323923   0.57727                    
    2      1.220556   1.300368   0.585767                   
    3      1.276826   1.329014   0.580988                   
    4      1.228651   1.351145   0.577801                   
    5      1.17493    1.316388   0.57196                    
    6      1.105402   1.329681   0.573553                   


[1.3296809382281656, 0.5735528556767513]

Training loss and validation loss are getting closer and smaller. We are on right track.

In [138]:
log_preds, y = learn.TTA() # (5, 2044, 120), (2044,)
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

(0.5724907063197026, 1.3786886784273897)

In [139]:
len(data.val_ds.y), data.val_ds.y[:5]

(1883, array([5, 3, 3, 7, 3]))

In [140]:
num = 13
print(data.val_ds.y[num])
print(np.argmax(probs[num]))

print(probs[num])
data.val_ds.fnames[num]

7
7
[0.00005 0.00049 0.00009 0.00001 0.00291 0.00021 0.00006 0.99384 0.00068 0.00082 0.00015 0.0007 ]


'train/2130.jpg'

In [141]:
np.argmax(probs,axis = 1)

array([ 5,  3,  3, ..., 10,  0,  0])

In [198]:
predictions = pd.DataFrame([data.val_ds.y,np.argmax(probs,axis = 1),
                            [each[1] for each in np.argsort(-probs,axis = 1)],
                            data.val_ds.fnames]).T

In [199]:
np.argmax(probs[1])

3

In [200]:
print(probs[0])

[0.00396 0.00326 0.00461 0.00408 0.00967 0.94797 0.00064 0.00199 0.00702 0.00491 0.0012  0.01071]


In [210]:
predictions.loc[:,'acc1'] = predictions[0] == predictions[1]
predictions.loc[:,'acc2'] = predictions[0] == predictions[2]
((predictions.acc1) | (predictions.acc2)).sum()

1373

In [211]:
len(predictions)

1883

In [213]:
predictions.acc1.sum()/1883

0.5724907063197026

In [ ]:
learn.save('299_pre')

In [ ]:
learn.load('299_pre')

In [ ]:
learn.fit(1e-2, 1, cycle_len=2) # 1+1 = 2 epochs

In [ ]:
learn.save('299_pre')

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

This dataset is so similar to ImageNet dataset. Training convolution layers doesn't help much. We are not going to unfreeze.

## Create submission

https://youtu.be/9C06ZPF8Uuc?t=1905

In [ ]:
data.classes

In [ ]:
data.test_ds.fnames

In [ ]:
log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
probs = np.mean(np.exp(log_preds),0)
#accuracy_np(probs, y), metrcs.log_loss(y, probs) # This does not make sense since test dataset has no labels

In [ ]:
probs.shape # (n_images, n_classes)

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes

In [ ]:
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')

## Individual prediction

In [ ]:
fn = data.val_ds.fnames[0]
fn

In [ ]:
Image.open(PATH + fn).resize((150, 150))

In [ ]:
# Method 1.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
ds = FilesIndexArrayDataset([fn], np.array([0]), val_tfms, PATH)
dl = DataLoader(ds)
preds = learn.predict_dl(dl)
np.argmax(preds)

In [ ]:
learn.data.classes[np.argmax(preds)]

In [ ]:
# Method 2.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
im = val_tfms(open_image(PATH + fn)) # open_image() returns numpy.ndarray
preds = learn.predict_array(im[None])
np.argmax(preds)